In [0]:
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from functools import reduce
from pyspark.sql.functions import *
from pyspark.sql import functions as F
import os
from pyspark.sql import *
from pyspark.sql.functions import col, sum

In [0]:
dbutils.fs.rm("dbfs:/FileStore/UberPickUps/Silver_Tables/", True)

Out[2]: True

In [0]:
delta_table_path = 'dbfs:/FileStore/UberPickUps/bronze_tables/'

try:
    file_paths = dbutils.fs.ls(delta_table_path)
except AnalysisException as e:
    print(f"Error: {e}")
    file_paths = []

In [0]:
spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")

In [0]:
directory_path = "dbfs:/FileStore/UberPickUps/bronze_tables/"

file_paths = [file.path for file in dbutils.fs.ls(directory_path)]

dfs = {}

for i, file_path in enumerate(file_paths):
    if "_delta_log" in file_path:
        continue
    try:
        df = spark.read.parquet(file_path)
        
        category_names = df.select('Category').distinct().rdd.flatMap(lambda x: x).collect()
        
        for category_name in category_names:
            dfs[category_name] = df
        
    except Exception as e:
        print(f"Error reading table from {file_path}: {e}")
 

In [0]:
len(dfs)

Out[6]: 18

In [0]:
for table_name, df in dfs.items():
    print(f"No of rows '{df.count()}':")
    print(f"DataFrame for table '{table_name}':")
    df.show(2) 

No of rows '100':
DataFrame for table 'other_American_B01362':
+----------+-----------+--------------------+--------------------+
|      DATE|       Time|     PICK_UP_ADDRESS|            Category|
+----------+-----------+--------------------+--------------------+
|07-01-2014|12:00:00 AM| 874 E 139th St M...|other_American_B0...|
|07-01-2014|12:01:00 AM| 628 E 141st St M...|other_American_B0...|
+----------+-----------+--------------------+--------------------+
only showing top 2 rows

No of rows '100':
DataFrame for table 'other_Diplo_B01196':
+----------+-----------+--------------------+------------------+
|      Date|       Time|          PU_Address|          Category|
+----------+-----------+--------------------+------------------+
|07-01-2014|12:00:00 AM| 2396 Valentine A...|other_Diplo_B01196|
|07-01-2014|12:01:00 AM| 1859 Walton Ave ...|other_Diplo_B01196|
+----------+-----------+--------------------+------------------+
only showing top 2 rows

No of rows '100':
DataFrame for tab